In [1]:
from models import HuBERTMultiHead
import pandas as pd
import torch
import librosa
import matplotlib.pyplot as plt
import os
import numpy as np

In [2]:
csv_path = '/media/maximos/9C33-6BBD/data/melos_singers/features/merged_df_PCA.csv'
feats = pd.read_csv(csv_path, delimiter=',')

In [3]:
feats.head()

,names,Pitch,SpectralCentroid,SpectralSpread,SpectralSkewness,SpectralKurtosis,SpectralFlatness,SpectralCrest,SpectralSlope,SpectralDecrease,...,F1_Hz,F2_Hz,F3_HZ,F4_Hz,Rate,Depth,Regularity,x,y,z
0,bellou_a_1,294.792756,1988.705488,1236.076582,0.667607,2.230896,6.130028e-05,0.166799,8.356768e-08,0.034523,...,965.355185,1566.595113,3081.040174,3536.460589,0.628707,0.031815,0.095775,9.317695,0.621495,-1.054224
1,bellou_a_2,224.575582,1191.358861,663.951795,1.619319,6.378916,1.482950e-09,0.309095,2.968422e-06,0.019057,...,899.487838,1495.686633,2559.188944,3348.608822,1.938681,0.015600,0.041192,2.236056,-6.705590,-4.355881
2,bellou_a_3,215.427334,1630.110589,1201.959572,1.233063,3.561876,3.840608e-04,0.198526,-8.364649e-07,0.028277,...,978.654497,1669.361870,3239.718778,4173.536668,3.915128,0.061797,0.434528,7.544067,-0.588950,-0.268951
3,bellou_a_4,310.702620,1215.894880,740.676596,1.352175,4.917070,1.353494e-03,0.247989,-1.853757e-06,0.043431,...,949.317310,1712.827614,2029.906806,3230.486712,8.998951,0.029717,0.463241,7.945791,-0.237960,-2.450706
4,bellou_a_5,326.715388,1111.645409,576.713488,1.669873,6.963033,4.287648e-09,0.394933,4.004495e-06,0.021064,...,966.616790,1598.766626,2945.418491,3379.549015,4.379635,0.035155,0.176779,8.613728,-1.355972,-2.128227


In [4]:
n = feats['names'].str.split('_').str[0]

In [5]:
print(n)

0          bellou
1          bellou
2          bellou
3          bellou
4          bellou
          ...    
380    tsaousakis
381    tsaousakis
382    tsaousakis
383    tsaousakis
384    tsaousakis
Name: names, Length: 385, dtype: object


In [6]:
# add names as new column
feats.insert(1, 'name_class', n, True)

In [7]:
feats.head()

,names,name_class,Pitch,SpectralCentroid,SpectralSpread,SpectralSkewness,SpectralKurtosis,SpectralFlatness,SpectralCrest,SpectralSlope,...,F1_Hz,F2_Hz,F3_HZ,F4_Hz,Rate,Depth,Regularity,x,y,z
0,bellou_a_1,bellou,294.792756,1988.705488,1236.076582,0.667607,2.230896,6.130028e-05,0.166799,8.356768e-08,...,965.355185,1566.595113,3081.040174,3536.460589,0.628707,0.031815,0.095775,9.317695,0.621495,-1.054224
1,bellou_a_2,bellou,224.575582,1191.358861,663.951795,1.619319,6.378916,1.482950e-09,0.309095,2.968422e-06,...,899.487838,1495.686633,2559.188944,3348.608822,1.938681,0.015600,0.041192,2.236056,-6.705590,-4.355881
2,bellou_a_3,bellou,215.427334,1630.110589,1201.959572,1.233063,3.561876,3.840608e-04,0.198526,-8.364649e-07,...,978.654497,1669.361870,3239.718778,4173.536668,3.915128,0.061797,0.434528,7.544067,-0.588950,-0.268951
3,bellou_a_4,bellou,310.702620,1215.894880,740.676596,1.352175,4.917070,1.353494e-03,0.247989,-1.853757e-06,...,949.317310,1712.827614,2029.906806,3230.486712,8.998951,0.029717,0.463241,7.945791,-0.237960,-2.450706
4,bellou_a_5,bellou,326.715388,1111.645409,576.713488,1.669873,6.963033,4.287648e-09,0.394933,4.004495e-06,...,966.616790,1598.766626,2945.418491,3379.549015,4.379635,0.035155,0.176779,8.613728,-1.355972,-2.128227


In [8]:
feats.insert(2, 'singer_id', feats.groupby('name_class').ngroup(), True)

In [9]:
feats.head()

,names,name_class,singer_id,Pitch,SpectralCentroid,SpectralSpread,SpectralSkewness,SpectralKurtosis,SpectralFlatness,SpectralCrest,...,F1_Hz,F2_Hz,F3_HZ,F4_Hz,Rate,Depth,Regularity,x,y,z
0,bellou_a_1,bellou,0,294.792756,1988.705488,1236.076582,0.667607,2.230896,6.130028e-05,0.166799,...,965.355185,1566.595113,3081.040174,3536.460589,0.628707,0.031815,0.095775,9.317695,0.621495,-1.054224
1,bellou_a_2,bellou,0,224.575582,1191.358861,663.951795,1.619319,6.378916,1.482950e-09,0.309095,...,899.487838,1495.686633,2559.188944,3348.608822,1.938681,0.015600,0.041192,2.236056,-6.705590,-4.355881
2,bellou_a_3,bellou,0,215.427334,1630.110589,1201.959572,1.233063,3.561876,3.840608e-04,0.198526,...,978.654497,1669.361870,3239.718778,4173.536668,3.915128,0.061797,0.434528,7.544067,-0.588950,-0.268951
3,bellou_a_4,bellou,0,310.702620,1215.894880,740.676596,1.352175,4.917070,1.353494e-03,0.247989,...,949.317310,1712.827614,2029.906806,3230.486712,8.998951,0.029717,0.463241,7.945791,-0.237960,-2.450706
4,bellou_a_5,bellou,0,326.715388,1111.645409,576.713488,1.669873,6.963033,4.287648e-09,0.394933,...,966.616790,1598.766626,2945.418491,3379.549015,4.379635,0.035155,0.176779,8.613728,-1.355972,-2.128227


In [10]:
features_list = list(feats.columns)

In [11]:
task_labels_map = {}
for i in range(1, len(features_list)-3, 1):
    task_labels_map[features_list[i]] = 1
# add singer identification
task_labels_map['singer_id'] = feats['singer_id'].max()+1 # accounting for zero

In [12]:
print( task_labels_map )

{'name_class': 1, 'singer_id': 6, 'Pitch': 1, 'SpectralCentroid': 1, 'SpectralSpread': 1, 'SpectralSkewness': 1, 'SpectralKurtosis': 1, 'SpectralFlatness': 1, 'SpectralCrest': 1, 'SpectralSlope': 1, 'SpectralDecrease': 1, 'SpectralRollOff': 1, 'SpectralVariation': 1, 'SpectralFlux': 1, 'HarmonicSpectralDeviation': 1, 'Tristimulus_1': 1, 'Tristimulus_2': 1, 'Tristimulus_3': 1, 'HarmonicOddToEvenRatio': 1, 'Inharmonicity': 1, 'HarmonicEnergy': 1, 'NoiseEnergy': 1, 'Noisiness': 1, 'HarmonicToNoiseEnergy': 1, 'PartialsToNoiseEnergy': 1, 'F1_Hz': 1, 'F2_Hz': 1, 'F3_HZ': 1, 'F4_Hz': 1, 'Rate': 1, 'Depth': 1, 'Regularity': 1}


In [13]:
model = HuBERTMultiHead(task_labels_map=task_labels_map)

You are using a model of type hubert to instantiate a model of type wav2vec2. This is not supported for all configurations of models and can yield errors.
Some weights of Wav2Vec2Model were not initialized from the model checkpoint at facebook/hubert-base-ls960 and are newly initialized: ['encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'encoder.pos_conv_embed.conv.parametrizations.weight.original1']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [14]:
dev = model.dev
sr = model.sampling_rate

In [15]:
# bring test audio data
audio_folder = '/media/maximos/9C33-6BBD/data/melos_singers/Rebetika_vowels/train/Bellou/'
audio_paths = os.listdir( audio_folder )
print(audio_paths)

['Bellou_a_2.mp3', 'Bellou_a_3.mp3', 'Bellou_a_4.mp3', 'Bellou_a_5.mp3', 'Bellou_a_6.mp3', 'Bellou_a_7.mp3', 'Bellou_a_8.mp3', 'Bellou_e_2.mp3', 'Bellou_e_3.mp3', 'Bellou_e_4.mp3', 'Bellou_e_5.mp3', 'Bellou_i_2.mp3', 'Bellou_i_3.mp3', 'Bellou_i_4.mp3', 'Bellou_i_5.mp3', 'Bellou_i_6.mp3', 'Bellou_i_7.mp3', 'Bellou_u_2.mp3', 'Bellou_u_3.mp3', 'Bellou_u_4.mp3', 'Bellou_u_5.mp3', 'Bellou_w_2.mp3', 'Bellou_w_3.mp3', 'Bellou_w_4.mp3', 'Bellou_w_5.mp3', 'Bellou_w_6.mp3', 'Bellou_w_7.mp3', 'Bellou_w_8.mp3']


In [16]:
audio_vars = []
for audio_path in audio_paths:
    audio_var, _ = librosa.load(audio_folder + audio_path, sr=sr)
    audio_vars.append(audio_var)

In [17]:
y = model( audio_vars[:8], task_name='singer_id', output_attentions=True )

after base model


In [18]:
print(y)

SequenceClassifierOutput(loss=None, logits=tensor([[ 0.0060,  0.1792,  0.1533,  0.0062, -0.0905, -0.1964],
        [-0.0631,  0.0858,  0.2371, -0.1115, -0.0327, -0.0558],
        [-0.1288,  0.0887,  0.2744, -0.1882,  0.0205, -0.1487],
        [-0.1548,  0.0671,  0.2130, -0.2157, -0.0305, -0.0837],
        [ 0.0036,  0.0605,  0.2097, -0.0718, -0.1385, -0.1217],
        [ 0.0205,  0.1508,  0.1973, -0.0978, -0.1255, -0.1176],
        [-0.1572,  0.0497,  0.2737, -0.1943, -0.0434, -0.1117],
        [-0.0542,  0.0374,  0.1431, -0.0414,  0.0086, -0.1973]],
       device='cuda:0', dtype=torch.float16, grad_fn=<AddmmBackward0>), hidden_states=tensor([[[ 0.3157, -0.2424,  0.1790,  ..., -0.1886, -0.0207,  0.7231],
         [ 0.3601, -0.2878,  0.2349,  ..., -0.2147, -0.0420,  0.7510],
         [ 0.3726, -0.3110,  0.2360,  ..., -0.2200, -0.0332,  0.9111],
         ...,
         [ 0.2218, -0.2073, -0.0068,  ..., -0.1740, -0.1976,  0.9121],
         [ 0.2600, -0.2223, -0.0131,  ..., -0.0964, -0.1565,